# Lecture: Witness Sets

In [1]:
using HomotopyContinuation

So far, we've seen that numerical methods are very good at solving **zero-dimensional polynomial systems**

**Question**: What do we do if we are interested in a **positive-dimensional** set of solutions?

**Answer**: Slice it with enough hyperplanes until we have a zero-dimensional solution set.



<img src="PictureWitness1.png" width="300">

The data in such a *slice* is called a **witness set**. The number of points in such a slice is called the **degree** of the variety.

A witness set for an irreducible variety $X \subset \mathbb{C}^n$ is a triple 
$$W=(F,L,S)$$
where 
    
- $\color{blue}{F}$ is a polynomial system for which $X$ is a component, 
- $\color{green}{L}$ is a **generic** affine linear space of **complementary dimension**
- $\color{red}{S} = X \cap L$



<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
How is this a good representation of the variety $X$?
</p>


Given a witness set $W=(F,L,S)$ for $X$...

Certain information about $X$ become obvious:
- dimension (equals the codimension of $L$)
- degree (equals the number of points in $S$)

Certain algorithms become almost trivial

- Membership: is $p$ contained in $X$?

- Sampling: find $100000$ points on $X$

- Find a (pseudo) witness set for a projection of $X$

One major advantage (compared to symbolic representations) is that they can be computed fast. 

One major disadvantage is that it can be difficult to prove things (but not impossible)

<hr style="border:1px solid gray"> </hr>

## Constructing a witness set using homotopycontinuation.jl

In [2]:
@var x y z
TwistedCubic=System([y^2-x*z,x*y-z,x^2-y])

System of length 3
 3 variables: x, y, z

 -x*z + y^2
 -z + x*y
 -y + x^2

In [3]:
#Try the following without codimension: you get an error because it expects the dimension to be zero.
W_twisted=witness_set(TwistedCubic; codim=2)

Tracking 5 paths... 100%|███████████████████████████████| Time: 0:00:08
  # paths tracked:                  5
  # non-singular solutions (real):  3 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (0)


Witness set for dimension 1 of degree 3

In [4]:
#Degree of the twisted cubic
degree(W_twisted)

3

In [5]:
#Dimension of the twisted cubic
dim(W_twisted)

1

In [6]:
#Generic linear space used to slice the twisted cubic
W_twisted.L

2-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.016393177609032694 - 0.42748383750249985im 0.6861094207796764 + 0.576987722106746im 0.00586716989050845 + 0.11529718112311549im]
b:
Complex{Float64}[-0.09333134543880195 + 0.024831373086473114im]

In [7]:
#Witness points
solutions(W_twisted)

3-element Array{Array{Complex{Float64},1},1}:
 [0.4086652681007739 + 0.4882835139152848im, -0.07141348860958074 + 0.3990890262467557im, -0.224052804569441 + 0.12822379474797194im]
 [-0.0538359549201583 - 0.16001682246283042im, -0.022707073428935676 + 0.017229316881151823im, 0.003979437522016019 + 0.0027059570106096435im]
 [-5.648294211229424 + 5.353153760743246im, 3.246972310448262 - 60.472374797254204im, 305.37806566186936 + 358.9473065413324im]

## Some simple algorithms involving witness sets

### Moving the linear space (this is the fundamental operation)

**KEY OBSERVATION**

The projection from the incidence variety $$W_X=\{(x,L) | x \in X \cap L, \text{codim}(L) = \text{dim}(X), L \text{ generic}\} \to \text{linear spaces of complementary dimension}$$
is a branched cover of degree deg$(X)$, and it's monodromy group is the full symmetric group.

Consequence: we can move in the parameter space of linear spaces of complementary dimension

In [8]:
L2 = LinearSubspace([1 2 3],[5])

2-dim. affine linear subspace {x | Ax=b} with eltype Float64:
A:
[-0.2672612419124243 -0.5345224838248488 -0.8017837257372732]
b:
[-1.3363062095621219]

In [9]:
W_twisted2=witness_set(W_twisted,L2)

Tracking 3 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  3
  # non-singular solutions (real):  3 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (1)


Witness set for dimension 1 of degree 3

In [10]:
solutions(W_twisted2)

3-element Array{Array{Complex{Float64},1},1}:
 [0.9241438381955056 + 1.9730282377693424e-42im, 0.8540418336747209 + 2.8698592549372254e-42im, 0.7892574981516841 + 1.4349296274686127e-42im]
 [-0.7954052524310862 - 1.0820358328370792im, -0.5381320279484714 + 1.721313969514515im, 2.290556436109343 - 0.7868640353973171im]
 [-0.7954052524310862 + 1.0820358328370792im, -0.5381320279484714 - 1.721313969514515im, 2.290556436109343 + 0.7868640353973171im]

### Sampling

In [11]:
function sample(W::WitnessSet)
    n=length(solutions(W)[1]) #number of variables
    randomLinear = LinearSubspace(randn(ComplexF64,dim(W),n),randn(ComplexF64,dim(W)))
    newWitness=witness_set(W,randomLinear)
    s=solutions(newWitness)[1]
    return s
end

sample (generic function with 1 method)

In [12]:
MySamples=[sample(W_twisted) for i in 1:100]

100-element Array{Array{Complex{Float64},1},1}:
 [3.219283444437547 + 1.146049977043523im, 9.050355345748216 + 7.37891943518849im, 20.679048681540305 + 34.12699271177124im]
 [0.24142774752098134 - 1.0187695883940757im, -0.9796041169635796 - 0.49191849393771797im, -0.7376552170131595 + 0.8792281090728312im]
 [0.681400528080613 - 1.1686006084771878im, -0.9013207024647154 - 1.592570143463363im, -2.4752388413233346 - 0.03189421542800039im]
 [0.14542107665703 - 0.42634378544639057im, -0.16062173385266812 - 0.12399874461129581im, -0.07622387963955402 + 0.05044804709021446im]
 [-0.039269621655110075 + 0.5179537055841076im, -0.2667339379433729 - 0.04067969210630036im, 0.03154473809409271 - 0.13655835544475062im]
 [0.9119734920717234 - 0.3752171154834845im, 0.6909077664897472 - 0.684376126185105im, 0.3732999325321933 - 0.8833733048949685im]
 [0.9849610343577495 - 0.6499228305291767im, 0.5477485535600309 - 1.2802973268214684im, -0.2925834806842977 - 1.6170372696595328im]
 [0.08622728581615292 + 


[[*Sampling real algebraic varieties for topological data analysis*](https://arxiv.org/abs/1802.07716)]Emilie Dufresne, Parker B. Edwards, Heather A. Harrington, Jonathan D. Hauenstein


[[*Sampling algebraic varieties for sum of squares programs*](https://arxiv.org/abs/1511.06751)]Diego Cifuentes, Pablo A. Parrilo.

[[*Learning algebraic varieties from samples*](https://link.springer.com/article/10.1007/s13163-018-0273-6)]Paul Breiding, Sara Kališnik, Bernd Sturmfels & Madeleine Weinstein 

### Membership

In [13]:
function membership(W::WitnessSet,p)
    n=length(solutions(W)[1]) #number of variables
    A=randn(ComplexF64,dim(W),n)
    randomLinearThroughP = LinearSubspace(A,A*p)
    newWitness=witness_set(W,randomLinearThroughP)
    check=findfirst(x->x≈p,solutions(newWitness))
    if check==nothing
        return false
    else
        return true
    end
end
    

membership (generic function with 1 method)

In [14]:
membership(W_twisted,[2,4,8])

true

In [15]:
membership(W_twisted,[2,5,8])

false

In [16]:
[membership(W_twisted,s) for s in MySamples]

100-element Array{Bool,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [17]:
# You can also do membership of subvarieties
@var z
W1 = witness_set(System([x^2+y^2+z^2-1]))
W2 = witness_set(System([x^2+y^2+z^2-1, x^2*y^5-12*x*y]))

W3 = witness_set(System([x^2+y^2+z^2-1.00001, x^2*y^5-12*x*y]))



Tracking 14 paths... 100%|██████████████████████████████| Time: 0:00:02
  # paths tracked:                  14
  # non-singular solutions (real):  14 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         14 (0)


Witness set for dimension 1 of degree 14

In [18]:
function contains(W1::WitnessSet,W2::WitnessSet)
    return membership(W1,solutions(W2)[1])
end

contains (generic function with 1 method)

In [19]:
contains(W1,W2)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


true

In [20]:
contains(W1,W3)

false

## Pseudo witness sets

Both the sampling and membership algorithms (and most other algorithms involving witness sets) depend on one thing:


**the ability to move the linear space of the witness set and track the witness points**

Observation: we can do this for projections trivially

**Definition:** A pseudo witness set for an (irreducible) variety $X\subset \mathbb{C}^n$ is a quadruple 
$$(F,\pi,\pi^{-1}(L),S)$$
where 
- $\pi:\mathbb{C}^N \to \mathbb{C}^n$ (really this works with any map by factoring through the graph)
- $X$ is a component of $\pi(\mathcal{V}(F))$ of the same dimension as $\mathcal{V}(F)$
- $L$ is a linear space of complementary dimension (to $X$) in $\mathbb{C}^n$
- $S = \pi^{-1}(L) \cap \mathcal V(F)$ (note! this is in $\mathbb{C}^N$)

<img src="PictureWitness2.png" width="300">

As the picture above shows, $\pi^{-1}(L)$ may not be generic for $\mathcal{V}(F)$ even though $L$ is generic for $X$


**Warning** be careful about the following situations

<img src="PictureWitness3b.png" width="800">

[[*Witness sets of projections*](https://www3.nd.edu/~jhauenst/preprints/hsProjection.pdf)] Hauenstein

**In general** the term pseudo witness set refers to any situation where you are computing a linear section of a variety, but not directly on the equations in the variables of its affine space. 

So let's compute a witness set for the parabola as a projection of the twisted cubic

In [21]:
PseudoWitness_Parabola=witness_set(W_twisted,LinearSubspace([3,6,0]',[1]))

Witness set for dimension 1 of degree 2

In [22]:
ProjectedSolutions=[s[1:2] for s in solutions(PseudoWitness_Parabola)]

2-element Array{Array{Complex{Float64},1},1}:
 [0.22871355387816908 + 0.0im, 0.05230988972758215 + 0.0im]
 [-0.728713553878169 - 7.346839692639297e-40im, 0.5310234436057513 - 3.6734198463196485e-40im]

Now let's compute a witness set for the parabola directly, move the linear space before, and check if we get the same points

In [23]:
Witness_Parabola=witness_set(System([y-x^2])) #computes a witness set for the parabola
Witness_Parabola2=witness_set(Witness_Parabola,LinearSubspace([3,6]',[1])) #moves it to the space we used above

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:01
  # paths tracked:                  2
  # non-singular solutions (real):  2 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (2)


Witness set for dimension 1 of degree 2

In [24]:
solutions(Witness_Parabola2)

2-element Array{Array{Complex{Float64},1},1}:
 [-0.728713553878169 + 7.703719777548943e-34im, 0.5310234436057513 + 0.0im]
 [0.22871355387816908 + 0.0im, 0.05230988972758215 + 0.0im]

#### Witness sets and their connection to sparse systems
Given a collection of points $A=(1,a_2,\ldots,a_k) \in \mathbb{N}^n$ you can parametrize a variety 
$$X_A = \{(1,x^{a_1},x^{a_2},\ldots,x^{a_k}) | x \in \mathbb{C}^n\} \subset \mathbb{C}^{|A|}_z$$
that is, it is the image of the map 
$$\varphi_{A}: \mathbb{C}^n \to \mathbb{C}^{k+1}$$
$$x \xrightarrow{\varphi_{A}} (1,x^{a_1},x^{a_2},\ldots,x^{a_k})=(z_1,\ldots,z_{k+1})$$
You can compute a witness set for $X_A$ by pulling back linear forms on $\mathbb{C}^{k+1}_z$
i.e.
$$\lambda_0 + \lambda_1 z_1 + \cdots + \lambda_{k+1}z_{k+1} = 0 \iff \lambda_0 + \lambda_1 x^{a_1} + \cdots + \lambda_{k+1}x^{a_{k+1}} = 0$$

One can then solve for $n$ such linear equations, not in $\mathbb{C}^{k+1}$, but in $\mathbb{C}^n$. Then after you've solved these for $p_1,\ldots,p_d \in \mathbb{C}^n$, your witness points are $\{\varphi_{A}(p_i)\}_{i=1}^d\subset \mathbb{C}^{k+1}$.

**Question:** How many solutions do you expect?

Alternatively, one can compute witness sets of images of maps via their graphs
$$\Gamma = \{(x,\varphi(x)) \mid x \in \mathbb{C}^n\} \subset \mathbb{C}^n \times \mathbb{C}^{k+1}$$
$$= \{z_i - x^{a_i} = 0\}_{i=0}^{k}$$

In [25]:
@var x,y
@var a[1:4],b[1:4]
@var z[1:4]
A = [[0,0],[0,1],[1,0],[1,1]]
φ = [x^A[i][1]*y^A[i][2] for i in 1:4]
Γ = System([φ[i]-z[i] for i in 1:4])

System of length 4
 6 variables: x, y, z₁, z₂, z₃, z₄

 1 - z₁
 y - z₂
 x - z₃
 -z₄ + x*y

In [26]:
WΓ = witness_set(System(Γ))

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Witness set for dimension 2 of degree 2

In [27]:
WΓ.L

4-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.0019473695310501388 - 0.14479780921287602im 0.06382243723321433 - 0.3620362384290097im … -0.3000441259284361 + 0.07029733094242555im 0.7318203123100667 - 0.16960400602079123im; 0.2622807397555529 - 0.12050784194197038im 0.10962076223391226 + 0.004398405132780822im … 0.24511350865228343 - 0.2851372642519093im 0.2361625005263142 - 0.4397324813750679im]
b:
Complex{Float64}[-0.3103061738580066 - 0.14836674999065091im, 0.15471353422996836 - 0.4477991332101073im]

In [28]:
L=LinearSubspace([0 0 randn(ComplexF64)  randn(ComplexF64)  randn(ComplexF64)  randn(ComplexF64) ;
                  0 0 randn(ComplexF64)  randn(ComplexF64)  randn(ComplexF64)  randn(ComplexF64) ],
                  randn(ComplexF64,2))

4-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[6.436874427156893e-18 + 9.90983785745606e-18im 0.0 + 0.0im … 0.14035139799474536 - 0.056742072041883335im -0.24867808021886656 - 0.21108678041925277im; 8.535733263888857e-18 + 1.314111896959817e-17im 0.0 + 0.0im … 0.6230505037479527 - 0.25197351104786875im 0.13647485879168525 - 0.20300272145916787im]
b:
Complex{Float64}[-0.3012238921715202 - 0.21567804211491493im, 0.5387116912149885 - 0.05933382922298651im]

In [29]:
WXA = witness_set(WΓ,L)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Witness set for dimension 2 of degree 2

Membership/sampling can be easily coded for these pseudo-witness sets as well

# Witness supersets, monodromy, the trace test, and irreducible decomposition

## Witness supersets

Now we're going to try to compute a witness set for the twisted cubic after "squaring up"

In [44]:
#If we only take two of the generators, we get a complete intersection of the union of a cubic and a line
@var x,y,z
CompleteIntersection=System([y^2-x*z,x*y-z])
W_complete=witness_set(CompleteIntersection)

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  4
  # non-singular solutions (real):  4 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (0)


Witness set for dimension 1 of degree 4


<img src="PictureWitness4.png" width="200">

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Warning: W_complete is not a witness set for the twisted cubic.
</p>

A **witness superset** for $\mathcal{V}(f_1,\ldots,f_k)$ in codimension $m$ is the triple $W=(F,L,S)$
    
- $\color{blue}{F}$ is a polynomial system
- $\color{green}{L}$ is a generic affine linear space of dimension $m$
- $\color{red}{S} = X \cap L$


Let $X \subseteq \mathbb{C}^n$ be an (equidimensional) variety of codimension $m$.

Recall, we have the "branched cover" 


$$W_X=\{(x,L) | x \in X \cap L, \text{codim}(L) = \text{dim}(X), L \text{ generic}\}$$
$$\,\,\,\, \downarrow \pi $$
$$\text{linear spaces of complementary dimension}$$



**Recall**: If $X$ is irreducible then the monodromy group of $\pi$ is the full symmetric group.

**Corollary**: If $X=X_1 \cup \cdots \cup X_k$ is equidimensional, then the monodromy group of $\pi$  is the product of symmetric groups $\mathfrak{S}_{\textrm{deg}(X_1)} \times \cdots \times \mathfrak{S}_{\textrm{deg}(X_k)}$

We can use monodromy to construct witness sets

In [45]:
M=monodromy_solve(CompleteIntersection; dim=1) 

MonodromyResult
• return_code → :success
• 3 solutions
• 6 tracked loops
• random_seed → 0xe04ce6e9
• trace → 6.979696404043302e-17

In [46]:
#Trying this many times will give us either 1 or 3 solutions (possibly 2 if the heuristic stop didn't work)
[length(solutions(monodromy_solve(CompleteIntersection;dim=1))) for i in 1:5]

5-element Array{Int64,1}:
 3
 3
 3
 1
 3

## The trace test

**Monodromy**: allows us to find points on the same component

**The trace test**: tells us when we've found all points on the same component


**Definition:** The *trace* of a subset of points in $\mathbb{C}^n$ is their coordinate-wise average.

**Lemma**: Let $X$ be irreducible. The trace of a subset of witness points of $X$ moves linearly if and only if the subset is not proper.


<img src="PictureWitness5.png" width="400">
<img src="traceline.png" width="400">
<img src="nonlineartrace.png" width="400">

[[*Trace Test*](https://arxiv.org/abs/1608.00540)] Leykin, Rodriguez, and Sottile

Now let's try moving a witness set and verifying this ourselves

In [47]:
W_complete

Witness set for dimension 1 of degree 4

In [48]:
A=randn(ComplexF64,1,3)
b=rand(ComplexF64)
L₁ = LinearSubspace(A,[b])
L₂ = LinearSubspace(A,[b+1])
L₃ = LinearSubspace(A,[b+2]);

In [49]:
trace₁=sum(solutions(witness_set(W_complete,L₁)))
println("First trace: \n",trace₁)
trace₂=sum(solutions(witness_set(W_complete,L₂)))
println("Second trace: \n",trace₂)
trace₃=sum(solutions(witness_set(W_complete,L₃)))
println("Third trace: \n",trace₃)

First trace: 
Complex{Float64}[0.8489418162895646 + 0.33489065601985923im, 0.738210683924271 - 1.2341888040128246im, -1.0639398028921836 - 0.7381597585205142im]
Second trace: 
Complex{Float64}[1.7738811627615638 + 1.1788335843265478im, 0.7382106839242708 - 1.2341888040128244im, -3.233771858847363 + 0.020969082606896228im]
Third trace: 
Complex{Float64}[2.6988205092335638 + 2.022776512633236im, 0.7382106839242708 - 1.2341888040128246im, -5.403603914802542 + 0.7800979237343058im]


In [50]:
# here we check trace2 is the midpoint of trace1 and trace3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 4.440892098500626e-16 - 4.440892098500626e-16im
 2.220446049250313e-16 - 4.440892098500626e-16im
 8.881784197001252e-16 - 8.881784197001252e-16im

In [51]:
trace_test(W_complete)

5.450611680439437e-17

We can find out which solution is the one on the line by checking traces of subsets

In [54]:
I=[1,2,3]
trace₁=sum(solutions(witness_set(W_complete,L₁))[I])
trace₂=sum(solutions(witness_set(W_complete,L₂))[I])
trace₃=sum(solutions(witness_set(W_complete,L₃))[I])
# here we check trace2 is the midpoint of trace1 and trace 3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 5.551115123125783e-16 - 2.220446049250313e-16im
 2.220446049250313e-16 - 4.440892098500626e-16im
 8.881784197001252e-16 - 8.881784197001252e-16im

In [55]:
[membership(W_twisted,s) for s in solutions(W_complete)]

4-element Array{Bool,1}:
 1
 1
 1
 0

## A harder example - the power of pseudo-witness sets

A plane quartic is called *Luroth* if it goes through the $10$ intersection points of $5$ lines.


<img src="lurothquartic2.png" width="600">


The set of *Luroth* quartics is an algebraic variety in the space of quartics $\mathbb{C}^{15}$

It is parametrized by all quartics $f(x,y)$ of the form $$\sum_{i=1}^5 \prod_{j \neq i} \ell_j(x,y)$$
where $\ell_j(x,y) = a_{j,1}x+a_{j,2}y+a_{j,3}$

This gives a map $\varphi: \mathbb{C}^{15} \to \mathbb{C}^{15}$ whose image is a hypersurface called the *Luroth hypersurface*

In [56]:
@var a[1:5,1:3]
@var t
ℓ = [a[i,1]*x+a[i,2]*y+a[i,3] for i in 1:5]
Quartic = sum([prod(ℓ[j] for j in filter(x->x!=i,collect(1:5))) for i in 1:5])
E=expand(Quartic);
EC=exponents_coefficients(E,[x,y])[2]; #This is the map from C^15->C^15

In [57]:
#You may be tempted to do the following and project as before. Don't. The graph has much higher degree than the 
#   Luroth hypersurface: moving the linear space 'vertical' will make many points go to infinity

#@var q[1:15]
#Graph = vcat([q[i]-EC[i] for i in 1:15],sum(randn(ComplexF64,15).*q) +rand(ComplexF64))
#MS=monodromy_solve(System(Graph);dim=14, show_progress=true)


<img src="PictureWitness6.png" width="300">

In [58]:
#Instead, we will do a new kind of witness set using a fibre product
@var m[1:15],k[1:15]
ParametrizedLines = [m[i]+k[i]*t for i in 1:15]
CutTheA = a[1]-1
PseudoWitnessSystem = System(vcat(CutTheA,[EC[i]-ParametrizedLines[i] for i in 1:15]),parameters=[vec(m); vec(k)])

#Note: this map is hundreds - to - one. You want to decide if two points are unique based on their 
#   associated t-value

MS=monodromy_solve(PseudoWitnessSystem;show_progress=true,distance = dist(x,y)=abs(x[end]-y[end]))

Solutions found: 54 	 Time: 0:00:03
  tracked loops (queued):            378 (0)
  solutions in current (last) loop:  0 (0)
  generated loops (no change):       7 (5)


MonodromyResult
• return_code → :heuristic_stop
• 54 solutions
• 378 tracked loops
• random_seed → 0x0dc00034

In [59]:
mixed_volume(PseudoWitnessSystem) #This is why we use monodromy

Mixed volume:  113240 	 Time: 0:00:12

LoadError: InterruptException:

# Constructing witness sets for all dimensions

In [60]:
#Let's start with the union of a sphere, a line, and 20 points
@var x[1:3]
Sphere=x[1]^2+x[2]^2+x[3]^2-1
Hyperplane1 = sum(randn(Float64,3).*x)+1 
Hyperplane2 = sum(randn(Float64,3).*x)+1 
Hyperplane3 = sum(randn(Float64,2).*[Hyperplane1,Hyperplane2]);
Quad1 = [x; 1]'*randn(Float64,4,4)*[x; 1]
Quad2 = [x; 1]'*randn(Float64,4,4)*[x; 1]
Quad3 = [x; 1]'*randn(Float64,4,4)*[x; 1]

f1 = Sphere*Hyperplane1*Quad1
f2 = Sphere*Hyperplane2*Quad2
f3 = Sphere*Hyperplane3*Quad3;
#We should have only the sphere in dimension 2
#We will have only the line in dimension 1
#We will have 20 points in dimension 0
#  We will have the 8 intersection points of 3 quadrics in dimension 0
#  Along with Hyperplane[1] intersect Quad[2] intersect Quad[3] and the 3 permutations thereof (3*4=12)

In [61]:
WF0=solve(System([f1,f2,f3]))

Tracking 125 paths... 100%|█████████████████████████████| Time: 0:00:04
  # paths tracked:                  125
  # non-singular solutions (real):  20 (10)
  # singular endpoints (real):      105 (0)
  # total solutions (real):         125 (10)


Result with 125 solutions
• 125 paths tracked
• 20 non-singular solutions (10 real)
• 105 singular solutions (0 real)
• random_seed: 0xced9e708
• start_system: :polyhedral
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   1   │  105  │   0    │    105     │
╰───────┴───────┴────────┴────────────╯


In [62]:
WF1=witness_set(System([f1,f2,f3]),dim=1)

Tracking 125 paths... 100%|█████████████████████████████| Time: 0:00:00
  # paths tracked:                  125
  # non-singular solutions (real):  1 (0)
  # singular endpoints (real):      16 (0)
  # total solutions (real):         17 (0)


Witness set for dimension 1 of degree 1

In [63]:
WF2=witness_set(System([f1,f2,f3]),dim=2)

Tracking 125 paths... 100%|█████████████████████████████| Time: 0:00:00
  # paths tracked:                  125
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Witness set for dimension 2 of degree 2

# The basic cascade algorithm

Here's one way to get a witness set for many dimensional components. We follow


[[*Numerical homotopies to compute generic points on positive dimensional algebraic sets*](https://arxiv.org/abs/math/9906198)] Andrew J. Sommese, Jan Verschelde, 1999

1) We assume $f=(f_1,\ldots,f_n)$ is a system of n equations in n variables (otherwise we slice to this case)

2) We use slack variables $z_1,\ldots,z_n$ and write 

$$F_i = \begin{bmatrix} f_1 + \sum_{j=1}^i \lambda_{1,j} z_j  \\  f_2 + \sum_{j=1}^i \lambda_{2,j} z_j   \\ \vdots \\  f_n + \sum_{j=1}^i \lambda_{n,j} z_j \\ \ell_1(x) + z_1 \\ \ell_2(x)+z_2 \\ \vdots \\ \ell_i(x)+z_i  \\ z_{i+1} \\ \vdots \\ z_{n}\end{bmatrix}$$
This system has $2n$ equations in $2n$ variables, and for $i=n$, this has isolated solutions of multiplicity one

3) We solve $F_n=0$ and call these solutions $S_n$

4) For $i=1,\ldots,n$,

4a) Run a homotopy to solve for $F_{n-i}$ from $F_{n-i+1}$ using $S_{n-i+1}$ as a start system.

4b) Call these solutions $S_{n-i}$.

4c) The solutions with $z_{n-i+1}=0$ belong to the codimension $i$ witness superset. Remove these from $S_{n-i}$ 

5) Return witness supersets

In [64]:
@var z[1:3],t
F₃ = [f1+rand(ComplexF64)*z[1]+rand(ComplexF64)*z[2]+rand(ComplexF64)*z[3],
             f2+rand(ComplexF64)*z[1]+rand(ComplexF64)*z[2]+rand(ComplexF64)*z[3],
             f3+rand(ComplexF64)*z[1]+rand(ComplexF64)*z[2]+rand(ComplexF64)*z[3],
             rand(ComplexF64)+sum(randn(ComplexF64,3).*x)+z[1],
             rand(ComplexF64)+sum(randn(ComplexF64,3).*x)+z[2],
             rand(ComplexF64)+sum(randn(ComplexF64,3).*x)+z[3]]
E₃ = System(F₃);

$$E_3 = \begin{bmatrix} f_1 + \lambda_{1,1}z_1+\lambda_{1,2}z_2+\lambda_{1,3}z_3 \\
f_2 + \lambda_{2,1}z_1+\lambda_{2,2}z_2+\lambda_{2,3}z_3 \\
f_3 + \lambda_{3,1}z_1+\lambda_{3,2}z_2+\lambda_{3,3}z_3\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
b_0+b_1x_1+b_2x_2+b_3x_3+z_2\\
c_0+c_1x_1+c_2x_2+c_3x_3+z_3\\
\end{bmatrix}$$

In [65]:
Results3=solve(E₃)

Tracking 125 paths... 100%|█████████████████████████████| Time: 0:00:07
  # paths tracked:                  125
  # non-singular solutions (real):  49 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (0)


Result with 49 solutions
• 125 paths tracked
• 49 non-singular solutions (0 real)
• random_seed: 0x2a971ebf
• start_system: :polyhedral


In [66]:
S3=solutions(Results3);

$$E_3 = \begin{bmatrix} f_1 + \lambda_{1,1}z_1+\lambda_{1,2}z_2+\lambda_{1,3}z_3 \\
f_2 + \lambda_{2,1}z_1+\lambda_{2,2}z_2+\lambda_{2,3}z_3 \\
f_3 + \lambda_{3,1}z_1+\lambda_{3,2}z_2+\lambda_{3,3}z_3\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
b_0+b_1x_1+b_2x_2+b_3x_3+z_2\\
c_0+c_1x_1+c_2x_2+c_3x_3+z_3\\
\end{bmatrix} 
\to 
\begin{bmatrix} f_1 + \lambda_{1,1}z_1+\lambda_{1,2}z_2 \\
f_2 + \lambda_{2,1}z_1+\lambda_{2,2}z_2 \\
f_3 + \lambda_{3,1}z_1+\lambda_{3,2}z_2\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
b_0+b_1x_1+b_2x_2+b_3x_3+z_2\\
z_3\\
\end{bmatrix} =E_2
$$

In [67]:
F₂ = vcat([evaluate(f,z[3]=>0) for f in F₃[1:5]],z[3]);
E₂ = System(t*F₃+(1-t)*F₂,parameters=[t]);

In [68]:
Results2=solve(E₂,S3,start_parameters=[1],target_parameters=[0])

Tracking 49 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  49
  # non-singular solutions (real):  49 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         49 (0)


Result with 49 solutions
• 49 paths tracked
• 49 non-singular solutions (0 real)
• random_seed: 0xb10e993b


In [69]:
S2=solutions(Results2);

In [70]:
Witness_Dim_2 = filter(x->isapprox(x[5],0;atol=0.000001),S2)

2-element Array{Array{Complex{Float64},1},1}:
 [0.7018707169402918 + 0.5929979327265746im, -0.7590584516361861 + 0.9522330305313996im, 1.1242642704791799 + 0.272705140255334im, 6.8960065528548765e-31 - 1.4705777542463869e-30im, -2.0541089284288494e-30 + 5.2279359543194934e-31im, 0.0 + 0.0im]
 [-0.8893516640285404 - 0.3619144959559666im, -0.7222627590457537 + 0.2662128666004733im, -0.29249122995657356 + 0.4430683945956028im, -1.794696279620841e-31 - 2.0489311592985584e-31im, 1.640527965978756e-32 + 1.4728504499577834e-31im, 0.0 + 0.0im]

In [71]:
println(membership(WF2,Witness_Dim_2[1][1:3]))
println(membership(WF2,Witness_Dim_2[2][1:3]))

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)
true
true


In [72]:
filter!(s->s ∉ Witness_Dim_2, S2);
length(S2)

47

$$E_2 = 
\begin{bmatrix} f_1 + \lambda_{1,1}z_1+\lambda_{1,2}z_2 \\
f_2 + \lambda_{2,1}z_1+\lambda_{2,2}z_2 \\
f_3 + \lambda_{3,1}z_1+\lambda_{3,2}z_2\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
b_0+b_1x_1+b_2x_2+b_3x_3+z_2\\
z_3\\
\end{bmatrix}  \to 
\begin{bmatrix} f_1 + \lambda_{1,1}z_1 \\
f_2 + \lambda_{2,1}z_1 \\
f_3 + \lambda_{3,1}z_1\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
z_2\\
z_3\\
\end{bmatrix} =E_1
$$

In [73]:
F₁ = vcat([evaluate(f,[z[2],z[3]]=>[0,0]) for f in F₂[1:4]],[z[2],z[3]])
E₁ = System(t*F₂+(1-t)*F₁,parameters=[t]);

In [74]:
Results1=solve(E₁,S2,start_parameters=[1],target_parameters=[0])

Tracking 47 paths... 100%|██████████████████████████████| Time: 0:00:01
  # paths tracked:                  47
  # non-singular solutions (real):  41 (0)
  # singular endpoints (real):      6 (0)
  # total solutions (real):         47 (0)


Result with 47 solutions
• 47 paths tracked
• 41 non-singular solutions (0 real)
• 6 singular solutions (0 real)
• random_seed: 0xaa99a91e
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   1   │   6   │   0    │     6      │
╰───────┴───────┴────────┴────────────╯


In [75]:
S1=solutions(Results1);

In [76]:
Witness_Dim_1 = filter(x->isapprox(x[4],0;atol=0.000001),S1)

1-element Array{Array{Complex{Float64},1},1}:
 [0.18986272663753628 - 0.2626008853056397im, 0.6569591705148489 + 0.6605337328622339im, -0.9662561071662771 + 0.5686454548142731im, 5.300164599316718e-33 - 1.0783510530859381e-32im, 0.0 + 0.0im, 0.0 + 0.0im]

In [77]:
println(membership(WF1,Witness_Dim_1[1][1:3]))
println(membership(WF2,Witness_Dim_1[1][1:3]))

true
false


In [78]:
filter!(s->s ∉ Witness_Dim_1, S1);
length(S1)

40

$$E_1 = 
\begin{bmatrix} f_1 + \lambda_{1,1}z_1 \\
f_2 + \lambda_{2,1}z_1 \\
f_3 + \lambda_{3,1}z_1\\
a_0+a_1x_1+a_2x_2+a_3x_3+z_1\\
z_2\\
z_3\\
\end{bmatrix}
\to
\begin{bmatrix} f_1  \\
f_2  \\
f_3 \\
z_1\\
z_2\\
z_3\\
\end{bmatrix} =E_0
$$

In [79]:
F₀ = vcat([evaluate(f,[z[1],z[2],z[3]]=>[0,0,0]) for f in F₁[1:3]],[z[1],z[2],z[3]])
E₀ = System(t*F₁+(1-t)*F₀,parameters=[t]);

In [80]:
Result0=solve(E₀,S1,start_parameters=[1],target_parameters=[0])

Tracking 40 paths... 100%|██████████████████████████████| Time: 0:00:02
  # paths tracked:                  40
  # non-singular solutions (real):  20 (10)
  # singular endpoints (real):      20 (0)
  # total solutions (real):         40 (10)


Result with 40 solutions
• 40 paths tracked
• 20 non-singular solutions (10 real)
• 20 singular solutions (0 real)
• random_seed: 0xcead24dc
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   1   │  20   │   0    │     20     │
╰───────┴───────┴────────┴────────────╯


### Numerical irreducible decomposition (of equidimensional varieties)

One of the most important algorithms in numerical algebraic geometry is taking a polynomial systme $F$ and computing a witness set for every irreducible component of $\mathcal V(F)$. This is called a **numerical irreducible decomposition**. We will go through how it works for equidimensional varieties

#### Step 1: compute a witness superset 

$\mathcal V(F) \to (F,L,S)$

#### Step 2: 

Using monodromy, identify solutions as belonging to the same component until the trace test passes.

$p \in S \to S_p=\{q \in S \mid q \text{ was found via monodromy starting from } p\}$

#### Step 3: 

If $S \backslash \bigcup_{p \text{ done in step 2}} S_p \neq \emptyset$ pick some $q$ not identified and repeat step 2

#### Output: 

Set $W_p = (F,L,S_p)$ and output the union of all $W_p$

In [81]:
using Bertini, LinearAlgebra

In [82]:
#Hollow orthogonal 4x4 matrices
@var m12,m13,m14,m21,m23,m24,m31,m32,m34,m41,m42,m43
M=[[0,m12, m13, m14] [m21, 0, m23, m24,] [m31, m32, 0, m34] [m41, m42, m43, 0]]
HollowO4=vec((M*M')-LinearAlgebra.I)

16-element Array{Expression,1}:
 -1 + m21^2 + m31^2 + m41^2
          m32*m31 + m41*m42
          m21*m23 + m41*m43
          m21*m24 + m31*m34
          m32*m31 + m41*m42
 -1 + m12^2 + m32^2 + m42^2
          m13*m12 + m42*m43
          m12*m14 + m32*m34
          m21*m23 + m41*m43
          m13*m12 + m42*m43
 -1 + m13^2 + m23^2 + m43^2
          m13*m14 + m23*m24
          m21*m24 + m31*m34
          m12*m14 + m32*m34
          m13*m14 + m23*m24
 -1 + m14^2 + m24^2 + m34^2

[[*Regenerative cascade homotopies for solving polynomial systems*](https://www.sciencedirect.com/science/article/abs/pii/S0096300311008150)]Jonathan D.Hauenstein Andrew J.Sommese Charles W.Wampler

In [79]:
bertini(System(HollowO4); TrackType=1)

File path: /tmp/jl_O9eWwX

 Bertini(TM) v1.5.1
  (August 29, 2016)

 D.J. Bates, J.D. Hauenstein,
 A.J. Sommese, C.W. Wampler

(using GMP v6.1.2, MPFR v3.1.4)



NOTE: You have requested to use adaptive path tracking.  Please make sure that you have
setup the following tolerances appropriately:
CoeffBound: 2.176827000000e+00, DegreeBound: 2.000000000000e+00
AMPSafetyDigits1: 1, AMPSafetyDigits2: 1, AMPMaxPrec: 1024


Tracking regeneration codim 1 of 10: 2 paths to track.
Tracking path 0 of 2

Sorting codimension 1 of 10: 2 paths to sort.
Sorting 0 of 2

Preparing regeneration codim 2 of 10: 2 witness points to move.
Moving 0 of 2

Tracking regeneration codim 2 of 10: 4 paths to track.
Tracking path 0 of 4

Sorting codimension 2 of 10: 4 paths to sort.
Sorting 0 of 4

Preparing regeneration codim 3 of 10: 4 witness points to move.
Moving 0 of 4

Tracking regeneration codim 3 of 10: 8 paths to track.
Tracking path 0 of 8

Sorting codimension 3 of 10: 8 paths to sort.
Sorting 0 of 8

Prep

Calculating 40 of 80
Calculating 60 of 80

Using monodromy to decompose codimension 10.
Performing monodromy loops: 80 points left to classify
Performing monodromy loops: 66 points left to classify
Performing monodromy loops: 60 points left to classify
Performing monodromy loops: 40 points left to classify
Performing monodromy loops: 40 points left to classify
Performing monodromy loops: 24 points left to classify
Performing monodromy loops: 24 points left to classify

Using combinatorial trace test to decompose codimension 10.


************* Witness Set Decomposition *************

| dimension | components | classified | unclassified
-----------------------------------------------------
|   2       |   28       |   80       |  0
-----------------------------------------------------

************** Decomposition by Degree **************

Dimension 2: 28 classified components
-----------------------------------------------------
   degree 2: 16 components
   degree 4: 12 components

**

LoadError: ArgumentError: Cannot open 'finite_solutions': not a file

### Other things you can do with witness sets

Given a witness set $W$ for $X$ you can also...

- find a witness set for $X \cap \mathcal V(f)$
- recover equations for $X$
- compute the Newton polytope of $X$ (when $X$ is a hypersurface)
- describe the real points of $X$
- much more...

[[*General witness sets for numerical algebraic geometry*](https://arxiv.org/abs/2002.00180)] Frank Sottile


[[*Introduction to numerical algebraic geometry, Solving polynomial equations*](https://link.springer.com/chapter/10.1007/3-540-27357-3_8)] A.J. Sommese, J. Verschelde, and C.W. Wampler